In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re

In [2]:
# base_url = 'https://www.imdb.com'
# user_id = 'ur17402378'
# ratings_url = f'{base_url}/user/{user_id}/ratings'

# page = requests.get(ratings_url)
# soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
base_url = 'https://www.imdb.com'
user_ids = ['ur17402378', 'ur3608743']
movie_details = []

for user_id in user_ids:
    ratings_url = f'{base_url}/user/{user_id}/ratings'

    page = requests.get(ratings_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    while True:
        ratings = soup.find_all('div', {'class': 'lister-item-content'})
        for div in ratings:
            header = div.find('h3', {'class': 'lister-item-header'})
            title_year = header.find('span', {'class': 'lister-item-year'}).text.strip()
            # Remove parentheses
            year = re.sub(r'[()\s]+', '', title_year)
            # Remove alphabets
            year = re.sub(r'[a-zA-Z]+', '', year)

            title = header.find('a').text.strip() 

            link = div.find('a')['href']
            movie_url = 'https://www.imdb.com' + link
            movie_page = requests.get(movie_url)
            movie_soup = BeautifulSoup(movie_page.content, 'html.parser')

            #Description
            description_elem = div.find('p', {'class': ''})
            if description_elem is not None:
                description = description_elem.text.strip()
            else:
                description = ''

            #Duration
            full_elem = div.find('p', {'class':'text-muted text-small'}) 
            duration_elem = full_elem.find('span', {'class': 'runtime'})
            if duration_elem is not None:
                duration = duration_elem.text.strip()
            else:
                duration = ''

            credits = div.find_all('p', {'class': 'text-muted text-small'})
            credits_text = credits[1].text.strip()

            #Directors
            if '|' in credits_text:
                director_index = credits_text.find('Director:') + len('Director: ')
                span_index = credits_text.find('<span class="ghost">|</span>')
                stars_index = credits_text.find('Stars:') + len('Stars:')
                directors = credits_text[director_index:director_index+20].strip()
                directors = directors.split('|')[0].strip()
            else:
                directors = None


            #Stars
            if "Director:" in credits_text:
                director_index = credits_text.find('Director:') + len('Director:')
                stars_index = credits_text.find('Stars:') + len('Stars:')
                span_index = credits_text.find('<span class="ghost">|</span>')
                stars = credits_text[stars_index:].strip()
            else:
                stars_index = credits_text.find('Stars:') + len('Stars:')
                stars = credits_text[stars_index:].strip()

            #Genre 
            genre_header = div.find('p', {'class': 'text-small'})   
            genre_elem = genre_header.find('span', {'class': 'genre'})
            if genre_elem is not None:
                genre = genre_elem.text.strip()
            else:
                genre = ''

            #Votes
            votes_element = credits[2]
            votes_elem = votes_element.find('span', {'name': 'nv'})
            if votes_elem:
                votes = []
                for span in votes_elem.find_all('span'):
                    v = span.text.strip()
                    if 'Votes:' in span.previous_sibling:
                        votes.append(v)

            if votes_elem :
                votes = votes_elem['data-value']
            else:
                votes = 0

            #Movie Ratings
            movie_rating = div.find('div', {'class': 'ipl-rating-star--other-user'})
            rating = float(movie_rating.find('span', {'class': 'ipl-rating-star__rating'}).text.strip())


            #User Ratings
            user_rating_elem = div.find('div', {'class': 'ipl-rating-star'})
            user_rating = float(div.find('span', {'class': 'ipl-rating-star__rating'}).text.strip())


            #User review (Blank)
            user_review_elem = div.find('div', {'class': 'text'})
            if user_review_elem is not None:
                user_review = user_review_elem.text.strip()
            else:
                user_review = ''

            movie_details.append({'user_id': user_id, 'title': title, 'year': year, 'link': movie_url, 'description': description, 
                                  'directors': directors, 'stars': stars, 'duration': duration, 
                                  'genre': genre, 'votes': votes, 'rating': rating, 'user_rating': user_rating, 
                                  'user_review': user_review})

        #iterate through the list of ratings by this user - test for 1 user is okay so far
        next_link = soup.find('a', {'class': 'next-page'})
        if next_link:
            next_url = f"{base_url}{next_link['href']}"
            response = requests.get(next_url)
            soup = BeautifulSoup(response.content, 'html.parser')
        else:
            break

In [6]:
#When done will export to csv

df = pd.DataFrame(movie_details)
filename = 'user_details.csv'
df.to_csv(filename, index=False)